# Workbench Viewer Preview - Create shareable maps
The workbench viewer application is being developed to provide users of the platform with a fast, robust, and flexible way to visualize and explore source datasets and analysis results interactively.

The viewer is primarily intended to support two use cases. The first is to provide a customizable interface for exploring very large raster imagery datasets. The second is to share analysis results across teams and with stakeholders.

This example notebook demonstrates the steps needed to create a customized viewer instance with both a source data source as well as with an analysis result. These customized instances of the viewer application can then be shared outside of a Workbench environment with other Descartes Labs users simply by sharing the URL of the viewer instance.

At this stage of development, the workbench viewer is being released as a _Preview_ to provide customers with the opportunity to evaluate prototype functionality and provide feedback on the utility of this new capability. The workbench viewer is designed to access data through Workflows, which is also currently available in a beta release. It is important to note that, as these capabilities are still at a pre-release stage, we don’t currently guarantee backwards compatibility. As this development continues and the technology moves towards a full release, it is expected that the specifics of the implementation and interfaces will change. Please refer to the [Workflows API documentation](https://docs.descarteslabs.com/guides/workflows.html) for more details.

Read our getting started [user guide](https://docs.google.com/document/d/19kotJZxYRZzBB72fpsexZqnhxrt2u3IPBM2d0-cXA70/edit?usp=sharing) for more detailed instructions.

## Import packages

In [ ]:
# Keep logging quiet
import logging
logging.getLogger().setLevel(logging.INFO)
logging.captureWarnings(True)

In [ ]:
import descarteslabs.workflows as wf
from viewer import Map, ProductLayer

## Instantiate a Workflows map

In [ ]:
map = Map(
    height=700,
    center=(-37.2481, -69.8514), #geology and snow in South America
    zoom=12,
    title='Viewer Workflows Map',
)

## Set up dynamic Workflows parameters
The Workflows API exposes a way to [parameterize](https://docs.descarteslabs.com/descarteslabs/workflows/docs/types/identifier.html) models. Parameters are used to pass values to the workflow as it runs. In this instance, they will be used to re-configure the workflow within the Viewer interface.

Parameters set in this part of the code can be changed directly within the Viewer interface to customize the data returned by the workflow when interactively exploring results in Viewer. Parameters that are not set in this part of the code will not be available to change in the Viewer interface.

In [ ]:
# These parameters do not yet have values assigned to them,
# but we will use them to define configurable properties in the Viewer interface

# Note: You must use these exact variable names for Viewer to correctly display your parameters
vw_product_id = wf.parameter('vw_product_id', wf.Str)
vw_start_date = wf.parameter('vw_start_date', wf.Str)
vw_end_date = wf.parameter('vw_end_date', wf.Str)
vw_band_0 = wf.parameter('vw_band_0', wf.Str)
vw_band_1 = wf.parameter('vw_band_1', wf.Str)
vw_band_2 = wf.parameter('vw_band_2', wf.Str)
vw_thresholds = wf.parameter("vw_thresholds", wf.Dict[wf.Str, wf.Tuple[wf.Float, wf.Float]])

## Define a threshold masking function

In [ ]:
def apply_band_threshold(bandname, img):
    min, max = vw_thresholds[bandname]
    min_masked = img.mask(img < min)
    return min_masked.mask(min_masked > max)

## Adding Imagery Products to the Workbench Viewer
We’ll start by creating a parameterized Workflows ImageCollection to access Sentinel 2 imagery and create an instance of the Workbench Viewer Application to interact with the imagery through the workflow.

In [ ]:
swir = wf.ImageCollection.from_id(vw_product_id,
    start_datetime=wf.Datetime.from_string(vw_start_date),
    end_datetime=wf.Datetime.from_string(vw_end_date)).filter(lambda img: img.properties['cloud_fraction'] <= 0.05).min(axis='images').pick_bands([vw_band_0, vw_band_1, vw_band_2]);

swir_masked = swir.map_bands(apply_band_threshold)

swir_layer = swir_masked.visualize('Sentinel-2 SWIR/NIR',
    scales = [[0, 0.6], [0, 0.6], [0, 0.6]],
    vw_product_id='sentinel-2:L1C',
    vw_start_date='2019-02-10',
    vw_end_date='2019-03-10', 
    vw_band_0='swir2',
    vw_band_1='swir1',
    vw_band_2='nir',
    vw_thresholds={
        'swir2': [0, 0.35], 
        'swir1': [0, 0.35], 
        'nir': [0, 0.35]
    })

map.add_layer(ProductLayer(swir_layer))
map

## Adding an analysis result to the Workbench Viewer 
Using the same pattern we followed to add imagery to the workbench viewer, it is also possible to visualize analysis calculated via a workflow on-the-fly within the workbench viewer. In this example, we’re demonstrating relatively simple band math to visualize NDVI run on Sentinel-2 imagery.

Note that parameters are being set for the start and end date, but not for other inputs to the workflow definition. Setting these dates will allow those to be changed in the Viewer interface to expand or contract the date range being included in the analysis.

In [ ]:
s2 = wf.ImageCollection.from_id('sentinel-2:L1C',
    start_datetime=wf.Datetime.from_string(vw_start_date),
    end_datetime=wf.Datetime.from_string(vw_end_date)).filter(lambda img: img.properties['cloud_fraction'] <= 0.05).median(axis='images')

nir, red = s2.unpack_bands('nir red')
ndvi = (nir-red)/(nir+red)

ndvi_layer = ndvi.visualize('Sentinel-2 NDVI',
    scales=[[0, 0.4]],
    colormap='viridis',
    vw_start_date='2018-03-01',
    vw_end_date='2018-04-01')

map.add_layer(ProductLayer(ndvi_layer))
map

In [ ]:
# Save your map to the file explorer. Double click the file to open.
map.save('saved-map.vw.json')

## Share your map
Once the imagery or analysis is created and the viewer is configured as desired, the configured instance of the viewer can be shared with other Descartes Labs Platform users. 
The configured viewer can be shared by clicking the blue _Share_ button in the upper right-hand corner of the application. Any Descartes Labs platform user with the link can interact with the map in their browser. Imagery is requested, and analysis is run, on-the-fly as users browse in the map interface.

![share-interactive-map.jpg](https://cdn.descarteslabs.com/assets/imgs/share-interactive-map.jpg)

## Try these links for more pre-configured products and analysis
* [Landsat 8 Collection 1 Real-Time](https://apps.descarteslabs.com/viewer2/master/index.html?data=%7Ba%3A%5B33.579400117396915%2C-101.89787244752688%5D%2Cu%3A10.477196017034414%2Cv%3A'dark'%2Cw%3A'Landsat%208'%2Cb%3A%5B%7Bx%3A'Landsat%208%20Collection%201%20Real-Time'%2Cd%3A%7Bbands%3A%5B'red'%2C'green'%2C'blue'%5D%2Cdate_range%3A%5B'2020-05-01'%2C'2020-06-03T16%3A36%3A02.000Z'%5D%2Cxyz_channel%3A'v0-14'%2Cproduct_id%3A'landsat%3ALC08%3A01%3ART%3ATOAR'%2Cxyz_id%3A'3d9fe5f9a33b6305dd1c08fc6d5772384d633265926c70b6'%7D%2Cs%3A%7Bpixel_color%3A%7Bscales%3A%5B%5B0%2C0.4%5D%2C%5B0%2C0.4%5D%2C%5B0%2C0.4%5D%5D%2Ccolormap%3A'viridis'%2Cstretches%3A%5B%5B0%2C0.4%5D%2C%5B0%2C0.4%5D%2C%5B0%2C0.4%5D%5D%7D%2Cvisible%3Atrue%2Copacity%3A1%7D%2Cc%3A'product'%7D%5D%7D)
* [MODIS Raw NDVI](https://apps.descarteslabs.com/viewer2/master/index.html?data=%7Ba%3A%5B35.016484001110314%2C-106.82363109942878%5D%2Cu%3A8.74597114926943%2Cv%3A'light'%2Cw%3A'MODIS%20Raw%20NDVI'%2Cb%3A%5B%7Bx%3A'Max%20NDVI%20calculation%20based%20on%20MODIS'%2Cd%3A%7Bbands%3A%5B'ndvi'%5D%2Cdate_range%3A%5B'2020-04-08'%2C'2020-06-08T15%3A57%3A02.768Z'%5D%2Cxyz_channel%3A'v0-14'%2Cproduct_id%3A'modis%3A09%3Amax-ndvi%3A16-day%3Av8b'%2Cxyz_id%3A'21dd69da8c3d4c5af3e5d4f39771407d3ad241ebb473a6b7'%7D%2Cs%3A%7Bpixel_color%3A%7Bscales%3A%5B%5B0%2C1%5D%5D%2Ccolormap%3A'RdYlGn'%2Cstretches%3A%5B%5B0%2C1%5D%5D%7D%2Cvisible%3Atrue%2Copacity%3A1%7D%2Cc%3A'product'%7D%5D%7D)